In [ ]:
# 使用BERT/GPT实现对话模型

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, get_linear_schedule_with_warmup
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #使用GPU加速
#设置参数
MAX_SEQ_LENGTH = 512 #最大序列长度
BATCH_SIZE = 8 #批次大小
LEARNING_RATE = 2e-5 #学习率
NUM_TRAIN_EPOCHS = 2 #训练轮次
WARMUP_STEPS = 100 #预热步数

接下来，需要加载SQuAD数据集，并使用BertTokenizer将输入文本转换为tokens，然后将tokens转换为模型可以接受的输入格式。

In [ ]:
#加载训练数据集
train_data = torch.load('train_data.pth') #train_data包含了input_ids, input_mask, segment_ids和start_positions, end_positions
#使用BertTokenizer将输入文本转换为tokens
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#将tokens转换为模型可以接受的输入格式
train_dataset = TensorDataset(
    torch.tensor(train_data['input_ids'], dtype=torch.long),
    torch.tensor(train_data['input_mask'], dtype=torch.long),
    torch.tensor(train_data['segment_ids'], dtype=torch.long),
    torch.tensor(train_data['start_positions'], dtype=torch.long),
    torch.tensor(train_data['end_positions'], dtype=torch.long))
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

接下来，需要定义模型并进行训练。首先，使用BertForQuestionAnswering或GPT2DoubleHeadsModel定义模型。然后，定义优化函数和学习率调度器。最后，使用训练数据集进行训练，并在验证集上进行验证。

In [ ]:
#定义模型
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
#model = GPT2DoubleHeadsModel.from_pretrained('gpt2')
#将模型移动到GPU上
model.to(device)
#定义优化函数和学习率调度器
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_dataloader) * NUM_TRAIN_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_steps)
#定义训练函数
def train(model, optimizer, scheduler, dataloader):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(dataloader, desc="Training")):
        input_ids, input_mask, segment_ids, start_positions, end_positions = tuple(t.to(device) for t in batch)
        #前向传播
        outputs = model(input_ids, attention_mask=input_mask, token_type_ids=segment_ids, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print("Average training loss: {0:.2f}".format(avg_loss))
#定义验证函数
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for step, batch in enumerate(tqdm(dataloader, desc="Evaluating")):
            input_ids, input_mask, segment_ids, start_positions, end_positions = tuple(t.to(device) for t in batch)
            #前向传播
            outputs = model(input_ids, attention_mask=input_mask, token_type_ids=segment_ids, start_positions=start_positions, end_positions=end_positions)
            loss = outputs[0]
            total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print("Average validation loss: {0:.2f}".format(avg_loss))
#训练模型
for epoch in range(NUM_TRAIN_EPOCHS):
    print("Epoch {}/{}".format(epoch+1, NUM_TRAIN_EPOCHS))
    train(model, optimizer, scheduler, train_dataloader)
    evaluate(model, validation_dataloader)

## 使用中文进行建模：
首先，需要下载中文的SQuAD数据集和BERT-Chinese模型。可以通过以下链接进行下载：


中文SQuAD数据集：https://github.com/CLUEbenchmark/CLUE/blob/main/baselines/CLUEdataset_public/squad_zh/train.json

BERT-Chinese模型：https://huggingface.co/bert-base-chinese/tree/main
然后，需要导入必要的库和设置一些参数。

In [ ]:
#使用BertTokenizer将输入文本转换为tokens
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

#定义模型
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese')

GPT模型的预训练模型中，目前没有预置的中文数据。不过，可以使用中文语料库进行训练，以得到适合中文语言的GPT模型。以下是一个使用中文维基百科数据集进行训练的示例：

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
# 加载预训练模型 tokenizer 和 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
# 加载中文维基百科数据集，将其转换为 TextDataset 对象
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="./zhwiki.txt",
    block_size=128  # 设置 block_size，即每个样本的最大长度，这里设置为 128
)
# 创建 DataCollator 对象，用于处理数据
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)
# 设置训练参数
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
)
# 创建 Trainer 对象，用于训练模型
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)
# 开始训练
trainer.train()

## SQuAD数据结构，如果想要加载自己的数据进行训练，如何处理数据？

SQuAD（Stanford Question Answering Dataset）是一个常用的问答数据集，包含大量的问题和对应的答案，可以用于训练和评估问答模型。SQuAD数据集的结构如下：


数据集文件是一个JSON文件，包含两个字段：data和version。

data字段是一个数组，每个元素代表一篇文章。每篇文章包含一个title字段和一个paragraphs字段。

title字段是文章的标题，paragraphs字段是一个数组，每个元素代表文章中的一个段落。每个段落包含一个context字段和一个qas字段。

context字段是段落的文本内容，qas字段是一个数组，每个元素代表一个问题和对应的答案。每个问题和答案都包括一个id字段和一个text字段，其中id字段是问题或答案的唯一标识符，text字段是问题或答案的文本内容。
如果想要加载自己的数据进行训练，需要将数据集转换为与SQuAD数据集相同的JSON格式。具体步骤如下：


将数据集转换为JSON格式，每篇文章包含一个title字段和一个paragraphs字段，每个段落包含一个context字段和一个qas字段，每个问题和答案都包括一个id字段和一个text字段。

将JSON格式的数据集保存为一个文件，文件名可以自定义。

使用Hugging Face的datasets库加载数据集文件，并进行相应的预处理，例如tokenization（分词）、答案抽取等。示例代码如下：

In [ ]:
# 构建自定义数据参考：


'''
InputExample类已经自动处理了qas、context、answer_text、start_position和end_position等字段，
这些都是通过类的构造函数传入的，而不需要手动设置这些字段。
具体来说，qas字段是一个列表，每个元素代表一个问题和对应的答案；context字段表示问题所在的原始文本，
answer_text字段表示问题的答案文本，start_position和end_position字段为答案在原始文本中的起始位置和结束位置。
当我们将自定义的训练数据转换为InputExample对象时，只需要按照InputExample类的构造函数要求传入这些字段，
InputExample类就会自动处理这些字段，并将其转换为模型需要的特征格式。
因此，在自定义数据中不需要手动设置qas、context、answer_text、start_position和end_position等字段。
'''

from transformers import InputExample
example = InputExample(
    'example_id', # 样例ID
    'What is the capital of France?', # 问题
    'Paris is the capital of France.', # 原文本
    'Paris', # 答案文本
    6, # 答案在原文本中的起始位置
    11 # 答案在原文本中的结束位置
)

In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
# 加载自定义数据集
dataset = load_dataset('json', data_files='my_dataset.json')
# 加载预训练 tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# 定义数据处理函数
def prepare_dataset(examples):
    # 分词
    tokenized_examples = tokenizer(
        examples['context'],
        examples['question'],
        truncation=True,
        max_length=512,
        padding='max_length'
    )
    # 答案抽取
    for i, answer in enumerate(examples['answers']):
        start_char = answer['answer_start']
        end_char = start_char + len(answer['text'])
        tokenized_examples['start_positions'][i] = tokenized_examples.char_to_token(i, start_char)
        tokenized_examples['end_positions'][i] = tokenized_examples.char_to_token(i, end_char - 1)
    return tokenized_examples
# 对数据集进行预处理
dataset = dataset.map(prepare_dataset, batched=True)
# 加载评价指标
metric = load_metric('squad')

在上面的代码中，我们使用了Hugging Face的datasets库和transformers库，通过调用load_dataset函数和AutoTokenizer.from_pretrained函数，加载自定义数据集和预训练tokenizer，并根据需要定义了数据处理函数prepare_dataset，将数据集进行了分词和答案抽取等预处理。最后，我们还加载了评价指标metric，用于评估模型的性能。
需要注意的是，自定义的数据集应该与SQuAD数据集具有相同的结构，即每篇文章包含一个title字段和一个paragraphs字段，每个段落包含一个context字段和一个qas字段，每个问题和答案都包括一个id字段和一个text字段。另外，自定义的数据集可能需要根据具体的应用场景进行调整，例如改变最大长度、划分训练集和验证集等。

要使用load_metric函数评价自定义的数据集，需要先准备好自定义数据集的答案，然后使用评价指标对模型的预测结果和答案进行衡量。假设自定义数据集的答案保存在一个名为my_dataset_answers.json的文件中，文件格式与SQuAD数据集相同，可以使用如下代码进行评价：

In [ ]:
from datasets import load_dataset, load_metric
# 加载自定义数据集和预测结果
dataset = load_dataset('json', data_files='my_dataset.json')
with open('my_dataset_predictions.txt', 'r') as f:
    predictions = f.readlines()
# 加载评价指标
metric = load_metric('squad')
# 对每个问题和答案进行评价
for i, example in enumerate(dataset['train']):
    prediction = predictions[i].strip()
    answer = example['answers']['text'][0]
    metric.add(prediction=prediction, reference=answer)
# 输出评价结果
result = metric.compute()
print(result)

在上面的代码中，我们首先使用load_dataset函数加载自定义数据集。然后，我们假设模型的预测结果保存在一个名为my_dataset_predictions.txt的文件中，每行代表一个问题的预测答案。接下来，我们使用load_metric函数加载squad评价指标，并对每个问题和答案进行评价，将模型的预测结果prediction和真实答案reference传递给评价指标的add函数。最后，我们调用compute函数计算评价结果，并输出到控制台。需要注意的是，在实际应用中，我们可能需要对验证集或测试集进行评价，而不是训练集。此外，我们还可以使用不同的评价指标，例如exact_match和f1等。

根据一个段落的文字提出问题，答案要在给定的段落里找，模型实现方式

In [ ]:
from transformers import pipeline
# 加载预训练的问答模型
qa_pipeline = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')
# 定义问题和段落
question = 'What is the capital of France?'
paragraph = 'Paris is the capital and most populous city of France, with a population of 2,148,271 residents as of 2020. Since the 17th century, Paris has been one of Europe\'s major centres of finance, diplomacy, commerce, fashion, science and the arts.'
# 进行答案抽取
result = qa_pipeline(question=question, context=paragraph)
# 输出提取的答案
print(result['answer'])

BertForQuestionAnswering和GPT2DoubleHeadsModel模型也支持使用pipeline进行根据给定的问题和段落进行答案抽取。这是因为Hugging Face Transformers库中的pipeline函数可以自动根据模型的输入和输出类型选择相应的模型，包括BertForQuestionAnswering和GPT2DoubleHeadsModel等模型。以下是使用BertForQuestionAnswering模型进行答案抽取的示例代码：

In [ ]:
from transformers import pipeline, BertForQuestionAnswering, BertTokenizer
# 加载预训练的BertForQuestionAnswering模型和分词器
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
# 定义问题和段落
question = 'What is the capital of France?'
paragraph = 'Paris is the capital and most populous city of France, with a population of 2,148,271 residents as of 2020. Since the 17th century, Paris has been one of Europe\'s major centres of finance, diplomacy, commerce, fashion, science and the arts.'
# 使用pipeline进行答案抽取
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)
result = qa_pipeline(question=question, context=paragraph)
# 输出提取的答案
print(result['answer'])

In [ ]:
from transformers import pipeline, GPT2DoubleHeadsModel, GPT2Tokenizer
# 加载预训练的GPT2DoubleHeadsModel模型和分词器
model = GPT2DoubleHeadsModel.from_pretrained('microsoft/DialoGPT-medium')
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-medium')
# 定义问题和段落
question = 'What is the capital of France?'
paragraph = 'Paris is the capital and most populous city of France, with a population of 2,148,271 residents as of 2020. Since the 17th century, Paris has been one of Europe\'s major centres of finance, diplomacy, commerce, fashion, science and the arts.'
# 使用pipeline进行答案抽取
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)
result = qa_pipeline(question=question, context=paragraph)
# 输出提取的答案
print(result['answer'])

需要注意的是，这些模型在使用pipeline进行答案抽取时，其实是将问题和段落拼接成一个字符串，然后对整个字符串进行答案抽取。因此，对于一些需要对段落进行分段等处理的场景，可能需要手动调用模型的forward函数进行处理。